In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm 

from sklearn.neighbors import KernelDensity
import joblib
from sklearn.metrics import roc_auc_score
import os
import random
import xgboost
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce



Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
InputAOI=pd.read_csv('data-challenge-phme/data/AOI_training.csv.zip')
InputAOI['PinNumber']=InputAOI['PinNumber'].astype('Int64').astype(str)
InputAOI['FigureID2']=InputAOI['FigureID']
InputAOI['ComponentID2']=InputAOI['ComponentID']
InputAOI['Count_Pin']=InputAOI.groupby(["PanelID","FigureID2","ComponentID2"])["PinNumber"].transform('count')
InputAOI['Count_Pin_Figure']=InputAOI.groupby(["PanelID","FigureID2"])["PinNumber"].transform('count')
InputAOI['Count_Pin_Panel']=InputAOI.groupby(["PanelID"])["PinNumber"].transform('count')


InputAOI['Target']=InputAOI['OperatorLabel'].astype('category').cat.codes

InputSPI=pd.DataFrame()
for item in range(4):
    InputSPI=pd.concat([InputSPI,pd.read_csv('data-challenge-phme/data/SPI_training_'+str(item)+'.csv.zip')])
InputAOI['key_spi']=(InputAOI['PanelID'].astype(str)+'_'+InputAOI['FigureID'].astype(str)+'_'+InputAOI['ComponentID'].astype(str))
InputSPI=InputSPI.dropna(subset=['FigureID'])

InputSPI['FigureID2']=InputSPI['FigureID']
InputSPI['ComponentID2']=InputSPI['ComponentID']
InputSPI['key_spi']=(InputSPI['PanelID'].astype(str)+'_'+InputSPI['FigureID'].astype(int,errors='ignore').astype(str)+'_'+InputSPI['ComponentID'].astype(str))
Features_to_bin=['Volume(%)',  'Area(%)', 'OffsetX(%)','OffsetY(%)',]


In [57]:
InputAOI["Count_Pin_Figure"].unique()

array([ 8,  4,  1,  3,  2,  5,  6,  9,  7, 10, 94, 11, 13, 24, 12, 14, 15,
       75, 20, 19, 62, 98, 26, 22])

In [58]:
InputAOI["Count_Pin_Figure"].nunique()

24

In [4]:
InputAOI.columns

Index(['PanelID', 'FigureID', 'MachineID', 'ComponentID', 'PinNumber',
       'AOILabel', 'OperatorLabel', 'RepairLabel', 'FigureID2', 'ComponentID2',
       'Count_Pin', 'Count_Pin_Figure', 'Count_Pin_Panel', 'Target',
       'key_spi'],
      dtype='object')

In [5]:

aoi_outter=InputAOI.loc[InputAOI.PinNumber=='<NA>']
aoi_outter['FigureID_ComponentID']=(aoi_outter['FigureID'].astype(str)+'_'+aoi_outter['ComponentID'].astype(str)).astype("category").cat.codes
aoi_outter['FigureID_ComponentID_PinNumber']=(aoi_outter['FigureID'].astype(str)+'_'+aoi_outter['ComponentID'].astype(str)+'_'+aoi_outter['PinNumber'].astype(str)).astype("category").cat.codes
aoi_outter['FigureID_ComponentID_PinNumber_AOILabel']=(aoi_outter['FigureID'].astype(str)+'_'+aoi_outter['ComponentID'].astype(str)+'_'+aoi_outter['PinNumber'].astype(str)+aoi_outter['AOILabel'].astype(str)).astype("category").cat.codes
aoi_outter['FigureID_ComponentID_AOILabel']=(aoi_outter['FigureID'].astype(str)+'_'+aoi_outter['ComponentID'].astype(str)+"_"+aoi_outter['AOILabel'].astype(str)).astype("category").cat.codes


/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set o

In [6]:
aoi_outter=aoi_outter.loc[~aoi_outter.RepairLabel.isna()]
aoi_outter=aoi_outter.loc[~(aoi_outter.RepairLabel=='NotYetClassified')]

In [7]:

Num_features=['Shape(um)',
   'PosX(mm)', 'PosY(mm)','SizeX', 'SizeY']

In [8]:
InputSPI['Shape(um)']=pd.to_numeric(InputSPI['Shape(um)'])

In [9]:
InputSPI=InputSPI.loc[InputSPI['key_spi'].isin(aoi_outter['key_spi'].unique().tolist())]

In [10]:
InputSPI_grouped=InputSPI.groupby(["PanelID","FigureID2","ComponentID2"])[Features_to_bin+Num_features].mean().reset_index()

In [11]:
aoi_outter=aoi_outter.merge(InputSPI_grouped,how="left",left_on=["PanelID","FigureID2","ComponentID2"],right_on=["PanelID","FigureID2","ComponentID2"])

In [12]:
aoi_outter.columns

Index(['PanelID', 'FigureID', 'MachineID', 'ComponentID', 'PinNumber',
       'AOILabel', 'OperatorLabel', 'RepairLabel', 'FigureID2', 'ComponentID2',
       'Count_Pin', 'Count_Pin_Figure', 'Count_Pin_Panel', 'Target', 'key_spi',
       'FigureID_ComponentID', 'FigureID_ComponentID_PinNumber',
       'FigureID_ComponentID_PinNumber_AOILabel',
       'FigureID_ComponentID_AOILabel', 'Volume(%)', 'Area(%)', 'OffsetX(%)',
       'OffsetY(%)', 'Shape(um)', 'PosX(mm)', 'PosY(mm)', 'SizeX', 'SizeY'],
      dtype='object')

In [13]:
Features_to_bin=['Volume(%)',  'Area(%)', 'OffsetX(%)','OffsetY(%)']

In [14]:
dict_var_info=joblib.load("dict_var_bin_mean_info")
for var in Features_to_bin[:]:
    mean_=dict_var_info[var+'_mean']
    std_=dict_var_info[var+'_std']
    aoi_outter[var]=norm.cdf(aoi_outter[var], loc=mean_, scale=std_)


In [15]:
alpha_list=[0.1,1e-6]
Features_alpha=[]
for var in Features_to_bin[:]:
    for alpha in alpha_list:
        aoi_outter[var+str(alpha)+"sup"]=aoi_outter[var]>(1-alpha)
        aoi_outter[var+str(alpha)+"inf"]=aoi_outter[var]<(alpha)
        Features_alpha.append(var+str(alpha)+"sup")
        Features_alpha.append(var+str(alpha)+"inf")




In [16]:
aoi_outter.columns

Index(['PanelID', 'FigureID', 'MachineID', 'ComponentID', 'PinNumber',
       'AOILabel', 'OperatorLabel', 'RepairLabel', 'FigureID2', 'ComponentID2',
       'Count_Pin', 'Count_Pin_Figure', 'Count_Pin_Panel', 'Target', 'key_spi',
       'FigureID_ComponentID', 'FigureID_ComponentID_PinNumber',
       'FigureID_ComponentID_PinNumber_AOILabel',
       'FigureID_ComponentID_AOILabel', 'Volume(%)', 'Area(%)', 'OffsetX(%)',
       'OffsetY(%)', 'Shape(um)', 'PosX(mm)', 'PosY(mm)', 'SizeX', 'SizeY',
       'Volume(%)0.1sup', 'Volume(%)0.1inf', 'Volume(%)1e-06sup',
       'Volume(%)1e-06inf', 'Area(%)0.1sup', 'Area(%)0.1inf',
       'Area(%)1e-06sup', 'Area(%)1e-06inf', 'OffsetX(%)0.1sup',
       'OffsetX(%)0.1inf', 'OffsetX(%)1e-06sup', 'OffsetX(%)1e-06inf',
       'OffsetY(%)0.1sup', 'OffsetY(%)0.1inf', 'OffsetY(%)1e-06sup',
       'OffsetY(%)1e-06inf'],
      dtype='object')

In [17]:
list_var_to_encode=['FigureID',"ComponentID","PinNumber","AOILabel","FigureID_ComponentID","FigureID_ComponentID_PinNumber",
                    "FigureID_ComponentID_PinNumber_AOILabel","FigureID_ComponentID_AOILabel"]#+Features_alpha#+Features_to_bin

list_var_to_encode=['FigureID',"ComponentID","PinNumber","AOILabel","FigureID_ComponentID_AOILabel","Count_Pin"]+Features_alpha#+Features_to_bin

list_var_to_encode=["FigureID_ComponentID",'FigureID',"ComponentID","PinNumber","AOILabel","FigureID_ComponentID_AOILabel"]+Features_alpha#+Features_to_bin
list_var_to_encode=["AOILabel","ComponentID","FigureID_ComponentID"]+Features_alpha


for var in list_var_to_encode:
    aoi_outter[var]=aoi_outter[var].astype('category').cat.codes
    
    
    

In [18]:
aoi_outter['Target']=aoi_outter['RepairLabel'].astype('category').cat.codes
 

In [19]:
def frequency_encoding(column, df, df_test=None):
    frequencies = df[column].value_counts().reset_index()
    df_values = df[[column]].merge(frequencies, how='left', 
                                   left_on=column, right_on='index').iloc[:,-1].values
    if df_test is not None:
        df_test_values = df_test[[column]].merge(frequencies, how='left', 
                                                 left_on=column, right_on='index').fillna(1).iloc[:,-1].values
    else:
        df_test_values = None
    return df_values, df_test_values

In [59]:
list_panel_id=InputAOI.PanelID.unique().tolist()


def cross_validation_panel_id_creation(list_panel_id,n_fold=5):
    Fold_list=[]
    random.shuffle(list_panel_id)
    list_panel_id=joblib.load("list_panel_id")
    len_fold=(len(list_panel_id))//n_fold
    for fold in range(n_fold):
        Fold_list.append(list_panel_id[fold*len_fold:(fold+1)*len_fold])
    return(Fold_list)
Fold_list=cross_validation_panel_id_creation(list_panel_id)
#Final_df=Final_df.loc[Final_df['ComponentID2']!="BC1"]
f1_score_list=[]
list_confusion_matrix=[]
for Test_fold_indice in range(5):
    print(Test_fold_indice)
    Train_indice=[]
    count_indice_fit_pin_training=0
    for indices in range(len(Fold_list)):
        if indices!=Test_fold_indice:
            Train_indice=Train_indice+Fold_list[indices]
        else:
            Test_indices=Fold_list[indices]
            Train_indice=Train_indice+Fold_list[indices]

    Train_df=aoi_outter.loc[aoi_outter["PanelID"].isin(Train_indice)].reset_index(drop=True)
    Test_df=aoi_outter.loc[aoi_outter["PanelID"].isin(Test_indices)].reset_index(drop=True)
    X_train=Train_df
    X_val=Test_df
    y_train=Train_df['Target']
    y_val=Test_df['Target']
    y_train=Train_df['Target']
    y_train=Train_df['Target']
    y_val=Test_df['Target']
    
    
    encoder=ce.CatBoostEncoder()

    AE_train=encoder.fit_transform(X_train[list_var_to_encode].copy(),y_train)
    AE_train.columns=[x+"_encoded" for x in AE_train.columns.tolist()]
    X_train_final=pd.concat([X_train,AE_train],axis=1)
    joblib.dump(encoder,"Cat_boost_step3_outer")

    AE_val=encoder.transform(X_val[list_var_to_encode].copy())
    AE_val.columns=[x+"_encoded" for x in AE_val.columns.tolist()]
    X_val_final=pd.concat([X_val,AE_val],axis=1)
    #col_features=[x for x in X_train_final.columns.tolist() if "_encoded" in str(x)]#+[x for x in X_train_final.columns.tolist() if "_is_in_panel" in str(x)]
    #col_features=col_features#+[x for x in X_train_final.columns.tolist() if '_is_in_figure' in str(x)]
    col_features=Num_features+[x+"_encoded" for x in list_var_to_encode]+["Count_Pin","Count_Pin_Figure","Count_Pin_Panel"]#+[x for x in X_val_final.columns.tolist() if "_counts" in x]#
    X_train_final=X_train_final.replace(np.nan,0)
    X_val_final=X_val_final.replace(np.nan,0)
    X_train_final=X_train_final[col_features].to_numpy()
    X_val_final=X_val_final[col_features].to_numpy()

    ratio=np.sum(y_train==0)/np.sum(y_train==1)
    


    xgb_cl = xgboost.XGBClassifier(objective="binary:logistic",n_estimators =200,max_depth = 15,scale_pos_weight=ratio)
    model = xgboost.XGBClassifier(**Best_trial)
    xgb_cl.fit(X_train_final, y_train)
    joblib.dump(xgb_cl,"xgboost_step3_outter")

    res_train=xgb_cl.predict(X_train_final)
    res_test=xgb_cl.predict(X_val_final)

    X_train['Pred']=res_train
    X_val['Pred']=res_test
   
    
    y_train=X_train.groupby(["PanelID","FigureID","ComponentID"])['Target'].min()
    y_val=X_val.groupby(["PanelID","FigureID","ComponentID"])['Target'].min()


    res_train=X_train.groupby(["PanelID","FigureID","ComponentID"])['Pred'].min()
    res_test=X_val.groupby(["PanelID","FigureID","ComponentID"])['Pred'].min()




    print(confusion_matrix(y_train, res_train,labels=[0,1]))
    print(confusion_matrix(y_val,res_test,labels=[0,1]))

    print(f1_score(y_train, res_train,pos_label=0))
    print(f1_score(y_val, res_test,pos_label=0))

    f1_score_list.append(f1_score(y_val, res_test,pos_label=0))

0
[[ 38   0]
 [ 10 115]]
[[ 3  2]
 [ 2 31]]
0.8837209302325582
0.6
1
[[ 39   0]
 [ 10 125]]
[[ 4  0]
 [ 4 19]]
0.8863636363636364
0.6666666666666666
2
[[ 35   0]
 [ 10 115]]
[[ 8  0]
 [ 6 27]]
0.8750000000000001
0.7272727272727273
3
[[ 25   0]
 [  9 110]]
[[15  3]
 [ 9 30]]
0.8474576271186441
0.7142857142857143
4
[[ 35   0]
 [ 12 116]]
[[ 8  0]
 [ 2 28]]
0.8536585365853657
0.888888888888889


In [60]:
np.mean(f1_score_list)

0.7194227994227994

In [22]:
Num_features

['Shape(um)', 'PosX(mm)', 'PosY(mm)', 'SizeX', 'SizeY']

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm 

from sklearn.neighbors import KernelDensity
import joblib
from sklearn.metrics import roc_auc_score
import os
import random
import xgboost
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import make_scorer
import category_encoders as ce
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
import logging
from sklearn.model_selection import cross_val_score

In [24]:



def objective(trial: Trial,X,y) -> float:
    
    joblib.dump(study, 'study.pkl')

    param = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']), 
        'tree_method': trial.suggest_categorical('tree_method',['exact']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        #'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        'n_estimators': trial.suggest_categorical('n_estimators', [30,40,50,70,100,150,200]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,8,9,10,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'nthread' : -1,
        #'feval': trial.suggest_categorical('feval',[f1_eval]),
        'scale_pos_weight' : trial.suggest_categorical('scale_pos_weight', [ratio])
        

    }
    
    
    model = xgboost.XGBClassifier(**param,feval=f1_eval)
    
    return cross_val_score(model, X_train_final, y_train, cv=5,scoring=make_scorer(f1_score, pos_label=0)).mean()

In [25]:
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.
from sklearn.metrics import f1_score
import numpy as np

def f1_eval(y_pred, y_true):
    #y_true = dtrain.get_label()
    err = f1_score(y_true,y_pred,pos_label=0)
    return 'f1_err', err

In [26]:
y_train=Train_df['Target']


In [41]:
study = optuna.create_study(direction='maximize')
#study = optuna.create_study(direction='maximize', storage="sqlite:///xgb_optuna_home_credit.db", study_name="optuna_xgb_output_8")
study.optimize(lambda trial : objective(trial,X_train_final,y_train),n_trials= 500)

np.mean(f1_score_list)

/home/phme/agaffet/.local/lib/python3.7/site-packages/optuna/samplers/_tpe/parzen_estimator.py:188: RuntimeWarning: divide by zero encountered in true_divide
  coefficient = 1 / z / p_accept
/home/phme/agaffet/.local/lib/python3.7/site-packages/optuna/samplers/_tpe/parzen_estimator.py:188: RuntimeWarning: divide by zero encountered in true_divide
  coefficient = 1 / z / p_accept
/home/phme/agaffet/.local/lib/python3.7/site-packages/optuna/samplers/_tpe/parzen_estimator.py:188: RuntimeWarning: divide by zero encountered in true_divide
  coefficient = 1 / z / p_accept
/home/phme/agaffet/.local/lib/python3.7/site-packages/optuna/samplers/_tpe/parzen_estimator.py:188: RuntimeWarning: divide by zero encountered in true_divide
  coefficient = 1 / z / p_accept
/home/phme/agaffet/.local/lib/python3.7/site-packages/optuna/samplers/_tpe/parzen_estimator.py:188: RuntimeWarning: divide by zero encountered in true_divide
  coefficient = 1 / z / p_accept
/home/phme/agaffet/.local/lib/python3.7/site-

0.9042113442113442

In [42]:
Best_trial = study.best_trial.params

In [43]:
col_features

['Shape(um)',
 'PosX(mm)',
 'PosY(mm)',
 'SizeX',
 'SizeY',
 'AOILabel_encoded',
 'ComponentID_encoded',
 'FigureID_ComponentID_encoded',
 'Volume(%)0.1sup_encoded',
 'Volume(%)0.1inf_encoded',
 'Volume(%)1e-06sup_encoded',
 'Volume(%)1e-06inf_encoded',
 'Area(%)0.1sup_encoded',
 'Area(%)0.1inf_encoded',
 'Area(%)1e-06sup_encoded',
 'Area(%)1e-06inf_encoded',
 'OffsetX(%)0.1sup_encoded',
 'OffsetX(%)0.1inf_encoded',
 'OffsetX(%)1e-06sup_encoded',
 'OffsetX(%)1e-06inf_encoded',
 'OffsetY(%)0.1sup_encoded',
 'OffsetY(%)0.1inf_encoded',
 'OffsetY(%)1e-06sup_encoded',
 'OffsetY(%)1e-06inf_encoded',
 'Count_Pin',
 'Count_Pin_Figure',
 'Count_Pin_Panel']

In [44]:
model = xgboost.XGBClassifier(**Best_trial)

In [49]:
model.fit(X_train_final,y_train)
print(confusion_matrix(y_train, model.predict(X_train_final),labels=[0,1]))

cross_val_score(model, X_train_final, y_train, cv=5,scoring=make_scorer(f1_score, pos_label=0)).mean()

[[ 46   0]
 [ 17 320]]


0.671927146927147

In [50]:
model.fit(X_train_final,y_train)


XGBClassifier(alpha=5.870505593904491, colsample_bytree=0.9897488196980344,
              lambda=0.008265969644375992, learning_rate=0.02525924006520497,
              max_depth=7, n_estimators=200,
              scale_pos_weight=0.13649851632047477, tree_method='exact')

In [51]:
joblib.dump(model,"xgboost_step3_outter")


['xgboost_step3_outter']

In [48]:
X_train_final.shape

(383, 27)

In [73]:
X_val_final=X_val_final[col_features]


/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [85]:
Train_df=aoi_outter.loc[aoi_outter["PanelID"].isin(Train_indice)].reset_index(drop=True)
Test_df=aoi_outter.loc[aoi_outter["PanelID"].isin(Test_indices)].reset_index(drop=True)
X_train=Train_df
X_val=Test_df
y_train=Train_df['Target']
y_val=Test_df['Target']
y_train=Train_df['Target']
y_train=Train_df['Target']
y_val=Test_df['Target']

for column in list_var_to_encode:
    train_values, test_values = frequency_encoding(column, X_train, X_val)
    X_train[column+'_counts'] = train_values
    X_val[column+'_counts'] = test_values

encoder=ce.CatBoostEncoder()

AE_train=encoder.fit_transform(X_train[list_var_to_encode].copy(),y_train)
AE_train.columns=[x+"_encoded" for x in AE_train.columns.tolist()]
X_train_final=pd.concat([X_train,AE_train],axis=1)
joblib.dump(encoder,"Cat_boost_step3_outer")

AE_val=encoder.transform(X_val[list_var_to_encode].copy())
AE_val.columns=[x+"_encoded" for x in AE_val.columns.tolist()]
X_val_final=pd.concat([X_val,AE_val],axis=1)
#col_features=[x for x in X_train_final.columns.tolist() if "_encoded" in str(x)]#+[x for x in X_train_final.columns.tolist() if "_is_in_panel" in str(x)]
#col_features=col_features#+[x for x in X_train_final.columns.tolist() if '_is_in_figure' in str(x)]
col_features=Num_features+[x+"_encoded" for x in list_var_to_encode]+["Count_Pin","Count_Pin_Figure","Count_Pin_Panel"]#+[x for x in X_val_final.columns.tolist() if "_counts" in x]#
#X_train_final=X_train_final.replace(np.nan,0)
#X_val_final=X_val_final.replace(np.nan,0)


In [40]:
X_val_final.shape

(73, 27)

In [79]:
len(col_features)

27

In [83]:
X_train_final.shape

(383, 29)

In [87]:
X_train_final[col_features].columns

Index(['Shape(um)', 'PosX(mm)', 'PosX(mm)', 'PosY(mm)', 'PosY(mm)', 'SizeX',
       'SizeY', 'AOILabel_encoded', 'ComponentID_encoded',
       'FigureID_ComponentID_encoded', 'Volume(%)0.1sup_encoded',
       'Volume(%)0.1inf_encoded', 'Volume(%)1e-06sup_encoded',
       'Volume(%)1e-06inf_encoded', 'Area(%)0.1sup_encoded',
       'Area(%)0.1inf_encoded', 'Area(%)1e-06sup_encoded',
       'Area(%)1e-06inf_encoded', 'OffsetX(%)0.1sup_encoded',
       'OffsetX(%)0.1inf_encoded', 'OffsetX(%)1e-06sup_encoded',
       'OffsetX(%)1e-06inf_encoded', 'OffsetY(%)0.1sup_encoded',
       'OffsetY(%)0.1inf_encoded', 'OffsetY(%)1e-06sup_encoded',
       'OffsetY(%)1e-06inf_encoded', 'Count_Pin', 'Count_Pin_Figure',
       'Count_Pin_Panel'],
      dtype='object')

In [ ]:
X_train_final=X_train_final[col_features].to_numpy()
X_val_final=X_val_final[col_features].to_numpy()